In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [12]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [35]:
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load('patch_camelyon',split=['train', 'test'],shuffle_files=True,as_supervised=True,with_info=True)

In [36]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

network_3= Sequential([
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform', activation='relu', input_shape = [96, 96, 3]),
                    MaxPooling2D(2),
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Flatten(),
                    Dense(1028,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(512,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(1, activation = 'sigmoid'),
                    ])
network_3.compile(optimizer= optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['acc'])

In [37]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [38]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [39]:
with tf.device("/XPU:0"):
    history = network_3.fit( ds_train, epochs = 5, validation_data = ds_test)


Epoch 1/5


2024-01-16 09:50:29.960104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


2048/2048 [==============================] - ETA: 0s - loss: 0.4075 - acc: 0.8186

2024-01-16 09:55:22.198128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


2048/2048 [==============================] - 304s 145ms/step - loss: 0.4075 - acc: 0.8186 - val_loss: 0.5875 - val_acc: 0.7592
Epoch 2/5
2048/2048 [==============================] - 298s 145ms/step - loss: 0.2582 - acc: 0.8949 - val_loss: 0.4921 - val_acc: 0.7879
Epoch 3/5
2048/2048 [==============================] - 297s 145ms/step - loss: 0.1845 - acc: 0.9294 - val_loss: 0.5355 - val_acc: 0.7764
Epoch 4/5
2048/2048 [==============================] - 297s 145ms/step - loss: 0.1352 - acc: 0.9504 - val_loss: 0.8066 - val_acc: 0.7316
Epoch 5/5
2048/2048 [==============================] - 295s 144ms/step - loss: 0.0990 - acc: 0.9640 - val_loss: 0.6216 - val_acc: 0.8035


In [40]:
# Access accuracy from history
train_accuracy = history.history['acc']
validation_accuracy = history.history['val_acc']

# Print or use the accuracy values
print(f'Training Accuracy: {train_accuracy[-1]}')
print(f'Validation Accuracy: {validation_accuracy[-1]}')

Training Accuracy: 0.9639625549316406
Validation Accuracy: 0.80352783203125
